In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
import mne

# Participant Overview

Here are the participants, totaling 88 individuals:

- **Alzheimer's Disease (AD) Group**: 36 participants
- **Frontotemporal Dementia (FTD) Group**: 23 participants
- **Healthy Controls (CN) Group**: 29 participants

## Dataset Access

The dataset is available at:

- [OpenNeuro Dataset ds004504](https://openneuro.org/datasets/ds004504/versions/1.0.8)

## Participant Details

- **Mini-Mental State Examination (MMSE) Scores**:

  - AD Group: Average MMSE = 17.75 (SD = 4.5)
  - FTD Group: Average MMSE = 22.17 (SD = 8.22)
  - CN Group: MMSE = 30

- **Mean Ages**:

  - AD Group: Mean Age = 66.4 years (SD = 7.9)
  - FTD Group: Mean Age = 63.6 years (SD = 8.2)
  - CN Group: Mean Age = 67.9 years (SD = 5.4)

- **Recording Durations**:

  - AD Group: Total = 485.5 minutes
  - FTD Group: Total = 276.5 minutes
  - CN Group: Total = 402 minutes

These recordings were collected using a clinical EEG system with 19 scalp electrodes, following the 10–20 international system. Each recording lasted approximately 13.5 minutes for the AD group, 12 minutes for the FTD group, and 13.8 minutes for the CN group.
The EEG recordings were exported in .eeg format and are transformed to BIDS accepted .set format for the inclusion in the dataset. ([mdpi.com](https://www.mdpi.com/2306-5729/8/6/95))

For more detailed information, please refer to the dataset description:

- [Dataset Description on GitHub](https://github.com/OpenNeuroDatasets/ds004504/blob/main/dataset_description.json)


In [9]:
participantsInfo = pd.read_table('./data/participants.tsv')
print(participantsInfo.head())


  participant_id Gender  Age Group  MMSE
0        sub-001      F   57     A    16
1        sub-002      F   78     A    22
2        sub-003      M   70     A    14
3        sub-004      F   67     A    20
4        sub-005      M   70     A    22


In [12]:
male_count = (participantsInfo['Gender'] == 'M').sum()
female_count = (participantsInfo['Gender'] == 'F').sum()
print(male_count)
print(female_count)

44
44
